In [ ]:
import paramiko
import subprocess

ip = "172.16.16.222"
user = "root"
pw = "asd123!@"
cmd = ""

def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)  # ← 여기가 핵심
    out = stdout.read().decode()
    err = stderr.read().decode()
    print(out if out else err)
    cli.close()

def dhcp_install():
    print("dhcp_install 시작")
    
    # tty 문제 없게 noninteractive 모드로 설치
    cmd = (
        'DEBIAN_FRONTEND=noninteractive apt update -y && '
        'DEBIAN_FRONTEND=noninteractive apt install isc-dhcp-server -y'
    )
    
    # cc() 함수를 이용해 명령 실행
    out = cc(cmd)
    print(out)
    print("dhcp_install 완료")


# DHCP 서버 설정 업데이트
def set_dhcp_file(NI, ip_subnet, netmask, exclusion_start, exclusion_end, gateway, domain, pool):
    # /etc/default/isc-dhcp-server 인터페이스 설정
    ch_NI_cmd = f"sed -i 's|^INTERFACESv4=.*$|INTERFACESv4=\"{NI}\"|' /etc/default/isc-dhcp-server"
    cc(ch_NI_cmd)
    print("/etc/default/isc-dhcp-server 파일 업데이트 완료")

    # dhcpd.conf 설정
    dhcp_conf_cmd = f"""
echo 'subnet {ip_subnet} netmask {netmask} {{
    range {exclusion_start} {exclusion_end};
    option routers {gateway};
    option subnet-mask {netmask};
    option domain-name-servers {domain}, 8.8.8.8;
    option domain-name "{pool}";
}}' >> /etc/dhcp/dhcpd.conf
"""
    cc(dhcp_conf_cmd)
    print("/etc/dhcp/dhcpd.conf 파일 업데이트 완료")

# DHCP 요청 및 서버 실행
def dhcp_request():
    cc("DEBIAN_FRONTEND=noninteractive dhclient -v")
    
    cmd = """
systemctl start isc-dhcp-server
systemctl enable isc-dhcp-server   # 부팅 시 자동 실행
systemctl status isc-dhcp-server

dhclient -v && ip a
"""
    cc(cmd)

def main():
    # 필요하다면 인터페이스와 DHCP 풀 설정
 

    # DHCP 파일 설정 적용
    # set_dhcp_file(NI, ip_subnet, netmask, exclusion_start, exclusion_end, gateway, domain, pool)

    # DHCP 서버 요청 및 실행
    dhcp_request()
    print("모든 작업 완료")

if __name__ == "__main__":
    main()
